In [1]:
import os

try:
    snakemake
except NameError:
    from snakemk_util import load_rule_args
    
    os.chdir('../..')
    
    snakemake = load_rule_args(
        snakefile = os.getcwd() + "/workflow/Snakefile",
        rule_name = 'benchmark_lapa',
        root = '..',
    )

In [9]:
import os


os.listdir(snakemake.input['long_rna_gm12'])

OSError: [Errno 116] Stale file handle: '/data/homezvol2/mcelik/Projects/lapa/data/results/lapa/gm12878/long_rna_lapa'

In [5]:
snakemake.input['long_rna_gm12']

'/data/homezvol2/mcelik/Projects/lapa/data/results/lapa/gm12878/long_rna_lapa'

In [8]:
! ls data/results/lapa/gm12878/long_rna_lapa

ls: cannot access data/results/lapa/gm12878/long_rna_lapa: Stale file handle
